In [ ]:
!pip install requests beautifulsoup4 faiss-cpu sentence-transformers transformers

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def generate_response(relevant_chunks, query, model_name='google/flan-t5-base'):
    """
    Generate a concise and specific answer based on the provided context and query.
    """
    # Load model and tokenizer
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Ensure relevant chunks are concise and within model limits
    max_chunk_length = 300  # Adjust to avoid token overflow
    relevant_chunks = ' '.join(relevant_chunks.split()[:max_chunk_length])

    # Refined prompt to focus on how AI models use genetic markers for personalized treatments
    prompt = (
        f"Context: {relevant_chunks}\n\n"
        f"Question: {query}\n"
        f"Provide a detailed answer on how AI models use BRCA1 and TP53 mutations to predict cancer outcomes and identify personalized treatments, including specific treatment recommendations."
    )

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    # Generate response using controlled parameters
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=150,  # Lengthened slightly for detail
        top_k=50,  # Focus on high-probability words
        num_beams=5,  # Beam search for better coherence
        no_repeat_ngram_size=2,  # Avoid repetition
        do_sample=False  # Use beam search for deterministic output
    )

    # Decode and clean the output
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.strip()


# Example query and context on AI-genomic cancer research involving BRCA1 and TP53
query = "How do AI models use BRCA1 and TP53 mutations to predict cancer outcomes and identify personalized treatments, including specific treatment recommendations?"
relevant_chunks = """
Stanford University has developed AI models that specifically focus on genetic mutations like BRCA1 and TP53 to predict cancer outcomes. These mutations, which are linked to a higher risk of developing certain cancers, are analyzed using machine learning algorithms that interpret gene expression data and mutation profiles. By incorporating these genetic markers, AI models predict cancer progression and suggest personalized treatments. For example, patients with BRCA1 mutations might be recommended therapies like PARP inhibitors, while those with TP53 mutations may receive targeted therapies to address specific cancer pathways.
"""

response = generate_response(relevant_chunks, query)
print(response)